In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel,BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [2]:
!pip install simpletransformers streamlit

In [3]:
kb_df = pd.read_excel('/content/ClientABC _ ATB Financial_Knowledge Base.xlsx',"Data Sheet")
q_df = pd.read_excel('/content/Industry Standard Questionnaires.xlsx',header=None)

In [4]:
def preprocess(df):
    cols = list(df)[:2]
    df[cols] = df[cols].ffill(axis=0)
    df = df.iloc[4:]
    #df = df.dropna().reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

In [5]:
kb_df = preprocess(kb_df)
q_df = q_df.rename(columns={0: 'question'})
q_df.head()

,question
0,Do you restrict access to cardholder data by b...
1,Do you identify and authenticate access to sys...
2,Do you restrict physical access to cardholders...
3,Does your business use network segmentation to...
4,Do you install and maintain a firewall configu...


In [6]:
kb_df['Answer'] = kb_df['Answer'].fillna('unanswerable')
kb_df.head()

4,Section Heading,Control Heading,Original ID,Question Text,Answer,Notes/Comment,identifier
5,Service Overview,Service Scope Question,3.3,What technology languages/platforms/stacks/com...,"""ClientABC"" database, Java, JavaScript, GO. Cl...",NaN,527c246f-dc2e-4873-a1d3-64a8ce1dda1b
6,Service Overview,Service Hosting,4.1,Is your service run from your own:\n- A. Data ...,B. The Cloud,NaN,909e4666-14a8-4a3f-8122-3f978c32caab
7,Service Overview,Service Hosting,4.2,Data Centre Location(s) (relative to services ...,"""ClientABC"" is available within the Amazon We...",NaN,9fd246fc-5ece-45e2-a0f8-9c34e6f3a8a9
8,Service Overview,Service Hosting,4.3,Which cloud providers do you rely on?,Our data centers are hosted on the cloud via A...,NaN,9e7ff978-612d-40d2-a18d-059389e820bb
9,Service Overview,Service Hosting,4.4,Have you researched your cloud providers best ...,Yes,NaN,a0aa89b4-8bd4-4b8d-a887-8d3d68da63c3


In [7]:
section_label_encoder = LabelEncoder()
control_label_encoder = LabelEncoder()

kb_df['Section Heading Encoded'] = section_label_encoder.fit_transform(kb_df['Section Heading'])
kb_df['Control Heading Encoded'] = control_label_encoder.fit_transform(kb_df['Control Heading'])

reverse_section_label_mapping = dict(enumerate(section_label_encoder.classes_))
reverse_control_label_mapping = dict(enumerate(control_label_encoder.classes_))

class CustomDataset(Dataset):
    def __init__(self, questions, section_labels, control_labels, tokenizer, max_len):
        self.questions = questions
        self.section_labels = section_labels
        self.control_labels = control_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        section_label = self.section_labels[idx]
        control_label = self.control_labels[idx]

        encoding = self.tokenizer.encode_plus(
            question,
            max_length=self.max_len,
            add_special_tokens=True,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'section_labels': torch.tensor(section_label, dtype=torch.long),
            'control_labels': torch.tensor(control_label, dtype=torch.long),
        }

train_df, test_df = train_test_split(kb_df, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(section_label_encoder.classes_) + len(control_label_encoder.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_dataset = CustomDataset(
    train_df['Question Text'].values,
    train_df['Section Heading Encoded'].values,
    train_df['Control Heading Encoded'].values,
    tokenizer,
    max_len=128
)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 25

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        section_labels = batch['section_labels'].to(device)
        control_labels = batch['control_labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        section_logits = logits[:, :len(section_label_encoder.classes_)]
        control_logits = logits[:, len(section_label_encoder.classes_):]

        loss_fn = torch.nn.CrossEntropyLoss()
        section_loss = loss_fn(section_logits, section_labels)
        control_loss = loss_fn(control_logits, control_labels)
        loss = (section_loss + control_loss) / 2

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Loss: {total_loss / len(train_loader)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/25
Loss: 2.8853137898952403
Epoch 2/25
Loss: 2.71793210760076
Epoch 3/25
Loss: 2.5385308240322355
Epoch 4/25
Loss: 2.306800479584552
Epoch 5/25
Loss: 2.066044754170357
Epoch 6/25
Loss: 1.8382386755436024
Epoch 7/25
Loss: 1.6354548347757218
Epoch 8/25
Loss: 1.4652294605336291
Epoch 9/25
Loss: 1.2921198923537072
Epoch 10/25
Loss: 1.143654744675819
Epoch 11/25
Loss: 1.0272834675109133
Epoch 12/25
Loss: 0.9242236221090276
Epoch 13/25
Loss: 0.8379020887486478
Epoch 14/25
Loss: 0.7895138149565839
Epoch 15/25
Loss: 0.7120596290902889
Epoch 16/25
Loss: 0.6256768906370123
Epoch 17/25
Loss: 0.6009265000515795
Epoch 18/25
Loss: 0.5417224359005055
Epoch 19/25
Loss: 0.5039685130753415
Epoch 20/25
Loss: 0.4814125327987874
Epoch 21/25
Loss: 0.4281958969349557
Epoch 22/25
Loss: 0.3913832107122908
Epoch 23/25
Loss: 0.3757855283453109
Epoch 24/25
Loss: 0.3332609894110801
Epoch 25/25
Loss: 0.31463202041514376


In [8]:
def predict_section_control(model, tokenizer, question, threshold=0.5, max_len=128):
    model.eval()
    encoding = tokenizer.encode_plus(
        question,
        max_length=max_len,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    section_logits = logits[:, :len(section_label_encoder.classes_)]
    control_logits = logits[:, len(control_label_encoder.classes_):]

    section_probs = F.softmax(section_logits, dim=1)
    control_probs = F.softmax(control_logits, dim=1)

    section_confidence, section_pred = torch.max(section_probs, dim=1)
    control_confidence, control_pred = torch.max(control_probs, dim=1)

    if section_confidence.item() < threshold:
        section_label = "Unanswerable"
    else:
        section_label = reverse_section_label_mapping[section_pred.item()]

    if control_confidence.item() < threshold:
        control_label = "Unanswerable"
    else:
        control_label = reverse_control_label_mapping[control_pred.item()]

    return section_label, control_label

def process_question(question, model, tokenizer):
  section_pred, control_pred = predict_section_control(model, tokenizer, question)
  return section_pred, control_pred

q_df[['section_pred', 'control_pred']] = q_df['question'].apply(process_question, args=(model, tokenizer)).tolist()


In [9]:
q_df.head()

,question,section_pred,control_pred
0,Do you restrict access to cardholder data by b...,Data Protection & Access Control,Background Checks
1,Do you identify and authenticate access to sys...,Proactive Security,Confidentiality
2,Do you restrict physical access to cardholders...,Data Protection & Access Control,Background Checks
3,Does your business use network segmentation to...,Proactive Security,Confidentiality
4,Do you install and maintain a firewall configu...,Proactive Security,Confidentiality


In [10]:
model.save_pretrained("./sectioncontrol/model")

tokenizer.save_pretrained("./sectioncontrol/tokenizer")

('./sectioncontrol/tokenizer/tokenizer_config.json',
 './sectioncontrol/tokenizer/special_tokens_map.json',
 './sectioncontrol/tokenizer/vocab.txt',
 './sectioncontrol/tokenizer/added_tokens.json')

In [11]:
train_questions_answers = []
train_contexts = []

for index, row in kb_df.iterrows():
    if pd.isna(row['Notes/Comment']):
        context = f"{row['Answer']}"
    else:
        context = f"{row['Notes/Comment']} | {row['Answer']}"

    question_answer = {
        "context": context,
        "question": row['Question Text'],
        "answer": row['Answer']
    }

    train_questions_answers.append(question_answer)
    if context not in train_contexts:
        train_contexts.append(context)

train_data = []

for context in train_contexts:
    qas = []
    for qa in train_questions_answers:
        if qa["context"] == context:
            answer_start = context.find(qa["answer"])
            if answer_start != -1:
                qas.append({
                    "id": str(len(qas) + 1).zfill(5),
                    "is_impossible": False,
                    "question": qa["question"],
                    "answers": [
                        {
                            "text": qa["answer"],
                            "answer_start": answer_start,
                        }
                    ],
                })
    train_data.append({
        "context": context,
        "qas": qas,
    })

print(train_data[0])

{'context': '"ClientABC" database, Java, JavaScript, GO. Cloud infrastructure is provided by AWS, Azure, and GCP.  Cluster nodes run CentOS and host the "ClientABC" database. "ClientABC"  is written in Java, JavaScript, and Go.\n', 'qas': [{'id': '00001', 'is_impossible': False, 'question': 'What technology languages/platforms/stacks/components are utilized in the scope of the application? (AWS? MySQL? Ruby on Rails? Go? Javascript?)', 'answers': [{'text': '"ClientABC" database, Java, JavaScript, GO. Cloud infrastructure is provided by AWS, Azure, and GCP.  Cluster nodes run CentOS and host the "ClientABC" database. "ClientABC"  is written in Java, JavaScript, and Go.\n', 'answer_start': 0}]}]}


In [12]:
import json

with open('train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

In [13]:
train_args = {
    'overwrite_output_dir': True,
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 50,
    "evaluate_during_training_steps": 500,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":16,
    "train_batch_size": 16,
    "eval_batch_size": 16
}

In [14]:
model_qa = QuestionAnsweringModel("bert",
                               "bert-base-uncased",
                               args = train_args,
                               use_cuda=True)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"finetune-BERT-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
traindata, valdata = train_test_split(train_data, test_size=0.15, random_state=42)

In [17]:
model_qa.train_model(traindata, eval_data=valdata)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 74/74 [00:00<00:00, 197945.47it/s]


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Running Epoch 1 of 50:   0%|          | 0/5 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 43/43 [00:00<00:00, 964.19it/s]

add example index and unique id: 100%|██████████| 43/43 [00:00<00:00, 289959.92it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 50 of 50:   0%|          | 0/5 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 43/43 [00:00<00:00, 756.13it/s]

add example index and unique id: 100%|██████████| 43/43 [00:00<00:00, 292309.68it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

(250,
 {'global_step': [5,
   10,
   15,
   20,
   25,
   30,
   35,
   40,
   45,
   50,
   55,
   60,
   65,
   70,
   75,
   80,
   85,
   90,
   95,
   100,
   105,
   110,
   115,
   120,
   125,
   130,
   135,
   140,
   145,
   150,
   155,
   160,
   165,
   170,
   175,
   180,
   185,
   190,
   195,
   200,
   205,
   210,
   215,
   220,
   225,
   230,
   235,
   240,
   245,
   250],
  'correct': [29,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33,
   33],
  'similar': [4,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   

In [18]:
result, texts = model_qa.eval_model(valdata)
print(result)

add example index and unique id: 100%|██████████| 43/43 [00:00<00:00, 299096.31it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'correct': 33, 'similar': 0, 'incorrect': 0, 'eval_loss': -6.54296875}


In [27]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

model = QuestionAnsweringModel("bert", "/content/outputs/best_model")

classifications = []
unanswerable_questions = []

In [26]:
def get_context(section_heading, control_heading, df):
    filtered_df = df[(df['Section Heading'] == section_heading) | (df['Control Heading'] == control_heading)]
    notes = filtered_df['Notes/Comment'].dropna().tolist()

    a= ' '.join(notes) if notes else ''
    return a

def classify_question(question, section_heading, control_heading, df):
    context = get_context(section_heading, control_heading, df)
    if not context.strip():
        return 'unanswerable'
    to_predict = [
        {
            "context": context,
            "qas": [
                {
                    "question": question,
                    "id": "0",
                }
            ],
        }
    ]

    answers, probabilities = model.predict(to_predict, n_best_size=2)
    print(answers)
    print(probabilities)
    if answers[0]['answer'] == '':
        return 'unanswerable'
    elif probabilities[0]['probability'][0] > 0.6:
        return 'answerable'
    else:
        return 'ambiguous'


In [28]:
classifications = []
unanswerable_questions = []

for _, row in q_df.iterrows():
    question = row['question']
    section_heading = row['section_pred']
    control_heading = row['control_pred']

    classification = classify_question(question, section_heading, control_heading, kb_df)
    classifications.append(classification)

    if classification == 'unanswerable':
        unanswerable_questions.append(question)



add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 7781.64it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['All']}]
[{'id': '0', 'probability': [0.838117441299773]}]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6831.11it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ["Our Customer's data will reside in the cloud and not onsite. The data our customers store in is arbitrary and unstructured. Also, we do not inspect this data. Therefore", 'Also, we do not inspect this data. Therefore']}]
[{'id': '0', 'probability': [0.4800297475659502, 0.4562618266919518]}]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11428.62it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Officer. In']}]
[{'id': '0', 'probability': [0.8017030553199359]}]


In [29]:
total_questions = len(classifications)
answerable_questions = classifications.count('answerable')
completion_percentage = (answerable_questions / total_questions) * 100

print(f"Completion Percentage: {completion_percentage:.2f}%")
print(f"\nTotal Questions: {total_questions}")
print(f"Answerable Questions: {answerable_questions}")
print(f"Unanswerable Questions: {classifications.count('unanswerable')}")
print(f"Ambiguous Questions: {classifications.count('ambiguous')}")

print("\nUnanswerable Questions:")
for q in unanswerable_questions:
    print(f"- {q}")

Completion Percentage: 26.92%

Total Questions: 26
Answerable Questions: 7
Unanswerable Questions: 6
Ambiguous Questions: 13

Unanswerable Questions:
- Do you track and monitor all access to network resources and cardholder data?
- Which types of facilities, such as retail outlets, corporate offices, data centers, call centers, and so on are included in the Payment Card Industry Data Security Standard (PCI DSS) review, and could you provide a summary of their locations?
- Can you list services that were included in the scope of the Payment Card Industry Data Security Standard (PCI DSS) Assessment?
- Can you list services that are provided by the service provider but were not included in the scope of the Payment Card Industry Data Security Standard (PCI DSS) Assessment?
- Does your company have a relationship with a Qualified Integrator and Reseller (QIR) for the purpose of the services being validated? If yes, can you provide the name of the Qualified Integrator and Reseller (QIR) comp

In [23]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [24]:
!streamlit run app.py &>/content/logs.txt &

In [25]:
!npx localtunnel --port 8501

your url is: https://quiet-laws-sort.loca.lt
^C
